### Predicting stock price movements

This program creates a data set to be used in machine learning models
to predict the future direction of stock prices. The general approach used in prediction is based on a paper by Jim Kyung-Soo Liew and Boris Mayster "Forecasting ETFs with Machine Learning Algorithms" (2017). 

This program gets the raw data from yahoo, transforms that data into a dataset that can directly be fed into scikit for prediction, and writes the transformed dataset in a CSV file.

The inputs to the program are (1) set of ticker symbols, and (2) timeframe, i.e, the beginning and end dates

We also need N (number of days to be used to calculate returns and average volumnes) and j (number of lags of N-day returns and average volumes.


In this program, N = 20 and j = 10

The same time frame will be used for all tickers and all tickers need to have data for the specified timeframe 

Initial version: February 21, 2017 (APPL data only)
Next revision: June 22, 2017  (Generalizes to a set of tickers)
This version: July 7, 2017 - creates an output in "wide" format so that for each day, there are a number of columns per Ticker. The ticker is part of the column name so that it's easy to identify tickers.

Note: yahoo data portal may not be available in the future. I had to do the "workaround" below - fix_yahoo_finance to get it to work in July 2017

Murat Aydogdu

In [1]:
from IPython.display import display
import pandas as pd
import numpy as np
import datetime

In [2]:
pd.set_option('display.max_columns', None)
#pd.options.display.max_rows = 1000
pd.options.display.float_format = '{:20,.2f}'.format

In [3]:
from pandas_datareader import data as pdr
import fix_yahoo_finance  # This is the "temporary" fix to get the data from yahoo
tickers = ["AAPL", "GOOGL","AMZN","MSFT","IBM"]

dt = pdr.get_data_yahoo(tickers, start="1999-12-01", end="2017-06-30")

[*********************100%***********************]  5 of 5 downloaded

In [4]:
# This is panel data, not a data frame
dt

<class 'pandas.core.panel.Panel'>
Dimensions: 6 (items) x 4424 (major_axis) x 5 (minor_axis)
Items axis: Open to Volume
Major_axis axis: 1999-12-01 00:00:00 to 2017-06-30 00:00:00
Minor_axis axis: AAPL to MSFT

In [5]:
# Convert panel to data frame
dtdf = dt.to_frame()
display(dtdf)

Open                 High  \
Date       minor                                             
1999-12-01 AAPL                  3.61                 3.73   
           AMZN                 87.25                87.88   
           IBM                 102.56               104.44   
           MSFT                 45.53                46.97   
1999-12-02 AAPL                  3.68                 3.95   
           AMZN                 86.00                91.31   
           IBM                 103.44               106.31   
           MSFT                 46.53                47.62   
1999-12-03 AAPL                  4.01                 4.13   
           AMZN                 92.50                93.38   
           IBM                 109.81               112.88   
           MSFT                 47.91                48.56   
1999-12-06 AAPL                  4.09                 4.19   
           AMZN                 86.88                89.88   
           IBM                 113.00               116.50   
           MSFT                 47.62                48.59   
1999-12-07 AAPL                  4.16                 4.21   
           AMZN                 88.00                88.00   
           IBM                 117.00               119.19   
           MSFT                 47.38                47.44   
1999-12-08 AAPL                  4.15                 4.21   
           AMZN                 86.81                93.00   
           IBM                 116.25               121.00   
           MSFT                 46.56                47.16   
1999-12-09 AAPL                  3.96                 3.96   
           AMZN                 95.50               113.00   
           IBM                 120.50               122.12   
           MSFT                 46.00                46.66   
1999-12-10 AAPL                  3.76                 3.90   
           AMZN                111.94               112.00   
...                               ...                  ...   
2017-06-23 AAPL                145.13               147.16   
           AMZN              1,002.54             1,004.62   
           GOOGL               975.50               986.62   
           IBM                 154.34               154.68   
           MSFT                 70.09                71.25   
2017-06-26 AAPL                147.17               148.28   
           AMZN              1,008.50             1,009.80   
           GOOGL               990.00               993.99   
           IBM                 154.71               155.75   
           MSFT                 71.40                71.71   
2017-06-27 AAPL                145.01               146.16   
           AMZN                990.69               998.80   
           GOOGL               961.60               967.22   
           IBM                 155.00               155.34   
           MSFT                 70.11                70.18   
2017-06-28 AAPL                144.49               146.11   
           AMZN                978.55               990.68   
           GOOGL               950.66               963.24   
           IBM                 155.15               155.55   
           MSFT                 69.21                69.84   
2017-06-29 AAPL                144.71               145.13   
           AMZN                979.00               987.56   
           GOOGL               951.35               951.66   
           IBM                 155.35               155.74   
           MSFT                 69.38                69.49   
2017-06-30 AAPL                144.45               144.96   
           AMZN                980.12               983.47   
           GOOGL               943.99               945.00   
           IBM                 154.28               154.50   
           MSFT                 68.78                69.38   

                                  Low                Close  \
Date       minor                                             
19

In [6]:
# Save raw data to a CSV first
dtdf.to_csv('RAWDATA.CSV')

In [7]:
# Transform and keep only adjusted prices and dollar volume
# Use dollar volume so that we can compare across securities
dtdf['P'] = dtdf['Adj Close']
dtdf['V'] = dtdf['Volume'] * dtdf['Close'] 
display(dtdf)

Open                 High  \
Date       minor                                             
1999-12-01 AAPL                  3.61                 3.73   
           AMZN                 87.25                87.88   
           IBM                 102.56               104.44   
           MSFT                 45.53                46.97   
1999-12-02 AAPL                  3.68                 3.95   
           AMZN                 86.00                91.31   
           IBM                 103.44               106.31   
           MSFT                 46.53                47.62   
1999-12-03 AAPL                  4.01                 4.13   
           AMZN                 92.50                93.38   
           IBM                 109.81               112.88   
           MSFT                 47.91                48.56   
1999-12-06 AAPL                  4.09                 4.19   
           AMZN                 86.88                89.88   
           IBM                 113.00               116.50   
           MSFT                 47.62                48.59   
1999-12-07 AAPL                  4.16                 4.21   
           AMZN                 88.00                88.00   
           IBM                 117.00               119.19   
           MSFT                 47.38                47.44   
1999-12-08 AAPL                  4.15                 4.21   
           AMZN                 86.81                93.00   
           IBM                 116.25               121.00   
           MSFT                 46.56                47.16   
1999-12-09 AAPL                  3.96                 3.96   
           AMZN                 95.50               113.00   
           IBM                 120.50               122.12   
           MSFT                 46.00                46.66   
1999-12-10 AAPL                  3.76                 3.90   
           AMZN                111.94               112.00   
...                               ...                  ...   
2017-06-23 AAPL                145.13               147.16   
           AMZN              1,002.54             1,004.62   
           GOOGL               975.50               986.62   
           IBM                 154.34               154.68   
           MSFT                 70.09                71.25   
2017-06-26 AAPL                147.17               148.28   
           AMZN              1,008.50             1,009.80   
           GOOGL               990.00               993.99   
           IBM                 154.71               155.75   
           MSFT                 71.40                71.71   
2017-06-27 AAPL                145.01               146.16   
           AMZN                990.69               998.80   
           GOOGL               961.60               967.22   
           IBM                 155.00               155.34   
           MSFT                 70.11                70.18   
2017-06-28 AAPL                144.49               146.11   
           AMZN                978.55               990.68   
           GOOGL               950.66               963.24   
           IBM                 155.15               155.55   
           MSFT                 69.21                69.84   
2017-06-29 AAPL                144.71               145.13   
           AMZN                979.00               987.56   
           GOOGL               951.35               951.66   
           IBM                 155.35               155.74   
           MSFT                 69.38                69.49   
2017-06-30 AAPL                144.45               144.96   
           AMZN                980.12               983.47   
           GOOGL               943.99               945.00   
           IBM                 154.28               154.50   
           MSFT                 68.78                69.38   

                                  Low                Close  \
Date       minor                                             
19

In [8]:
# Rename indices Date: level 0 and Ticker: level 1
dtdf.index = dtdf.index.set_names('Date', level=0) 
dtdf.index = dtdf.index.set_names('Ticker', level=1)

# Drop columns other than adjusted close and dollar volume
dtdf.drop(dtdf.columns[[0,1,2,3,4,5]], inplace = True, axis=1)  

# Sort by ticker
dtdf.sort_index(level=1, ascending = True, inplace=True)
display(dtdf)

,,P,V
Date,Ticker,,
1999-12-01,AAPL,3.32,"15,937,708,675.00"
1999-12-02,AAPL,3.55,"15,628,950,925.00"
1999-12-03,AAPL,3.71,"18,627,700,000.00"
1999-12-06,AAPL,3.74,"13,536,689,600.00"
1999-12-07,AAPL,3.80,"13,107,253,250.00"
1999-12-08,AAPL,3.55,"11,346,078,975.00"
1999-12-09,AAPL,3.40,"22,502,407,900.00"
1999-12-10,AAPL,3.32,"16,422,361,200.00"
1999-12-13,AAPL,3.19,"13,116,549,600.00"


Construct variables using lags and leads.

Today is day *t*. For each day *t*, we need P~t~, P~t-N~, P~t+N~, and AV~t~
 
 AV~t~ is the average dollar volume of last N days, ending (i.e., including) today. This is parallel to how returns are calculated.

In [9]:
# m : minus, p: plus
# Negative values can be used for leads ("forward lags")
# Return will be measured based on Pt and Pt-N
# Average dollar volume will be measured based on Vt through Vt-N

N = 20
lagN = N*1
leadN = N*-1

# Price: we just need the N-lead and N-lag values
cname = 'P'+'m'+str(N).zfill(2)
dtdf[cname] = dtdf.groupby('Ticker')['P'].shift(lagN)
cname = 'P'+'p'+str(N).zfill(2)
dtdf[cname] = dtdf.groupby('Ticker')['P'].shift(leadN)
cname = 'AV'
# Average dollar volume of last N days, ending (including) today
dtdf[cname] = dtdf.groupby('Ticker')['V'].rolling(lagN).mean().reset_index(0,drop=True)    
display(dtdf)

,,P,V,Pm20,Pp20,AV
Date,Ticker,,,,,
1999-12-01,AAPL,3.32,"15,937,708,675.00",nan,3.24,nan
1999-12-02,AAPL,3.55,"15,628,950,925.00",nan,3.32,nan
1999-12-03,AAPL,3.71,"18,627,700,000.00",nan,3.61,nan
1999-12-06,AAPL,3.74,"13,536,689,600.00",nan,3.31,nan
1999-12-07,AAPL,3.80,"13,107,253,250.00",nan,3.35,nan
1999-12-08,AAPL,3.55,"11,346,078,975.00",nan,3.06,nan
1999-12-09,AAPL,3.40,"22,502,407,900.00",nan,3.21,nan
1999-12-10,AAPL,3.32,"16,422,361,200.00",nan,3.15,nan
1999-12-13,AAPL,3.19,"13,116,549,600.00",nan,2.99,nan


In [10]:
# Compute the N-day return 
#  and the sign of the N-period ahead price movement

dtdf['R'] = (dtdf['P'] / dtdf['Pm20']) - 1
dtdf['Y'] = (dtdf['Pp20'] >= dtdf['P']).astype(int)

dtdf = dtdf.dropna()
display(dtdf)

,,P,V,Pm20,Pp20,AV,R,Y
Date,Ticker,,,,,,,
1999-12-30,AAPL,3.24,"5,194,783,125.00",3.32,3.28,"11,253,684,233.75",-0.03,1
1999-12-31,AAPL,3.32,"4,210,459,750.00",3.55,3.35,"10,682,759,675.00",-0.07,1
2000-01-03,AAPL,3.61,"14,993,938,575.00",3.71,3.23,"10,501,071,603.75",-0.03,0
2000-01-04,AAPL,3.31,"13,129,676,000.00",3.74,3.19,"10,480,720,923.75",-0.12,0
2000-01-05,AAPL,3.35,"20,236,361,600.00",3.80,3.33,"10,837,176,341.25",-0.12,0
2000-01-06,AAPL,3.06,"18,239,354,000.00",3.55,3.48,"11,181,840,092.50",-0.14,1
2000-01-07,AAPL,3.21,"11,460,768,200.00",3.40,3.68,"10,629,758,107.50",-0.05,1
2000-01-10,AAPL,3.15,"12,342,501,500.00",3.32,3.71,"10,425,765,122.50",-0.05,1
2000-01-11,AAPL,2.99,"10,238,412,800.00",3.19,3.63,"10,281,858,282.50",-0.06,1


In [11]:
pd.options.mode.chained_assignment = None  # default='warn'
# j-lags of returns and average volumes
J = 10
vars = ['R','AV']
for i in vars:
    for j in range (1,J+1):
        cname = i+str(j).zfill(2)
        dtdf[cname] = dtdf[i].shift(j)
display(dtdf)  

,,P,V,Pm20,Pp20,AV,R,Y,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,AV01,AV02,AV03,AV04,AV05,AV06,AV07,AV08,AV09,AV10
Date,Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,
1999-12-30,AAPL,3.24,"5,194,783,125.00",3.32,3.28,"11,253,684,233.75",-0.03,1,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan,nan
1999-12-31,AAPL,3.32,"4,210,459,750.00",3.55,3.35,"10,682,759,675.00",-0.07,1,-0.03,nan,nan,nan,nan,nan,nan,nan,nan,nan,"11,253,684,233.75",nan,nan,nan,nan,nan,nan,nan,nan,nan
2000-01-03,AAPL,3.61,"14,993,938,575.00",3.71,3.23,"10,501,071,603.75",-0.03,0,-0.07,-0.03,nan,nan,nan,nan,nan,nan,nan,nan,"10,682,759,675.00","11,253,684,233.75",nan,nan,nan,nan,nan,nan,nan,nan
2000-01-04,AAPL,3.31,"13,129,676,000.00",3.74,3.19,"10,480,720,923.75",-0.12,0,-0.03,-0.07,-0.03,nan,nan,nan,nan,nan,nan,nan,"10,501,071,603.75","10,682,759,675.00","11,253,684,233.75",nan,nan,nan,nan,nan,nan,nan
2000-01-05,AAPL,3.35,"20,236,361,600.00",3.80,3.33,"10,837,176,341.25",-0.12,0,-0.12,-0.03,-0.07,-0.03,nan,nan,nan,nan,nan,nan,"10,480,720,923.75","10,501,071,603.75","10,682,759,675.00","11,253,684,233.75",nan,nan,nan,nan,nan,nan
2000-01-06,AAPL,3.06,"18,239,354,000.00",3.55,3.48,"11,181,840,092.50",-0.14,1,-0.12,-0.12,-0.03,-0.07,-0.03,nan,nan,nan,nan,nan,"10,837,176,341.25","10,480,720,923.75","10,501,071,603.75","10,682,759,675.00","11,253,684,233.75",nan,nan,nan,nan,nan
2000-01-07,AAPL,3.21,"11,460,768,200.00",3.40,3.68,"10,629,758,107.50",-0.05,1,-0.14,-0.12,-0.12,-0.03,-0.07,-0.03,nan,nan,nan,nan,"11,181,840,092.50","10,837,176,341.25","10,480,720,923.75","10,501,071,603.75","10,682,759,675.00","11,253,684,233.75",nan,nan,nan,nan
2000-01-10,AAPL,3.15,"12,342,501,500.00",3.32,3.71,"10,425,765,122.50",-0.05,1,-0.05,-0.14,-0.12,-0.12,-0.03,-0.07,-0.03,nan,nan,nan,"10,629,758,107.50","11,181,840,092.50","10,837,176,341.25","10,480,720,923.75","10,501,071,603.75","10,682,759,675.00","11,253,684,233.75",nan,nan,nan
2000-01-11,AAPL,2.99,"10,238,412,800.00",3.19,3.63,"10,281,858,282.50",-0.06,1,-0.05,-0.05,-0.14,-0.12,-0.12,-0.03,-0.07,-0.03,nan,nan,"10,425,765,122.50","10,629,758,107.50","11,181,840,092.50","10,837,176,341.25","10,480,720,923.75","10,501,071,603.75","10,682,759,675.00","11,253,684,233.75",nan,nan


In [12]:
dtdf = dtdf.dropna()

In [14]:
# All we need are the returns and volumes, 
# plus the target variable Y (future N-day price movement)
dtdf = dtdf.drop(['P','V','Pm20','Pp20', 'AV', 'R'], 1)

In [15]:
# Convert Date and Ticker into columns
dtdf.reset_index(inplace=True)
display(dtdf)

,Date,Ticker,Y,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,AV01,AV02,AV03,AV04,AV05,AV06,AV07,AV08,AV09,AV10
0,2000-01-13,AAPL,1,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,-0.12,-0.03,-0.07,-0.03,"10,828,705,711.25","10,281,858,282.50","10,425,765,122.50","10,629,758,107.50","11,181,840,092.50","10,837,176,341.25","10,480,720,923.75","10,501,071,603.75","10,682,759,675.00","11,253,684,233.75"
1,2000-01-14,AAPL,1,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,-0.12,-0.03,-0.07,"11,322,248,551.25","10,828,705,711.25","10,281,858,282.50","10,425,765,122.50","10,629,758,107.50","11,181,840,092.50","10,837,176,341.25","10,480,720,923.75","10,501,071,603.75","10,682,759,675.00"
2,2000-01-18,AAPL,1,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,-0.12,-0.03,"11,242,355,241.25","11,322,248,551.25","10,828,705,711.25","10,281,858,282.50","10,425,765,122.50","10,629,758,107.50","11,181,840,092.50","10,837,176,341.25","10,480,720,923.75","10,501,071,603.75"
3,2000-01-19,AAPL,1,0.04,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,-0.12,"11,220,169,388.75","11,242,355,241.25","11,322,248,551.25","10,828,705,711.25","10,281,858,282.50","10,425,765,122.50","10,629,758,107.50","11,181,840,092.50","10,837,176,341.25","10,480,720,923.75"
4,2000-01-20,AAPL,1,0.09,0.04,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,-0.12,"11,668,364,487.50","11,220,169,388.75","11,242,355,241.25","11,322,248,551.25","10,828,705,711.25","10,281,858,282.50","10,425,765,122.50","10,629,758,107.50","11,181,840,092.50","10,837,176,341.25"
5,2000-01-21,AAPL,0,0.11,0.09,0.04,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,-0.14,"13,872,175,564.39","11,668,364,487.50","11,220,169,388.75","11,242,355,241.25","11,322,248,551.25","10,828,705,711.25","10,281,858,282.50","10,425,765,122.50","10,629,758,107.50","11,181,840,092.50"
6,2000-01-24,AAPL,1,0.11,0.11,0.09,0.04,0.02,-0.00,-0.08,-0.06,-0.05,-0.05,"14,153,621,956.89","13,872,175,564.39","11,668,364,487.50","11,220,169,388.75","11,242,355,241.25","11,322,248,551.25","10,828,705,711.25","10,281,858,282.50","10,425,765,122.50","10,629,758,107.50"
7,2000-01-25,AAPL,1,0.03,0.11,0.11,0.09,0.04,0.02,-0.00,-0.08,-0.06,-0.05,"14,442,203,396.89","14,153,621,956.89","13,872,175,564.39","11,668,364,487.50","11,220,169,388.75","11,242,355,241.25","11,322,248,551.25","10,828,705,711.25","10,281,858,282.50","10,425,765,122.50"
8,2000-01-26,AAPL,1,0.13,0.03,0.11,0.11,0.09,0.04,0.02,-0.00,-0.08,-0.06,"14,930,717,985.35","14,442,203,396.89","14,153,621,956.89","13,872,175,564.39","11,668,364,487.50","11,220,169,388.75","11,242,355,241.25","11,322,248,551.25","10,828,705,711.25","10,281,858,282.50"
9,2000-01-27,AAPL,1,0.12,0.13,0.03,0.11,0.11,0.09,0.04,0.02,-0.00,-0.08,"15,132,560,456.60","14,930,717,985.35","14,442,203,396.89","14,153,621,956.89","13,872,175,564.39","11,668,364,487.50","11,220,169,388.75","11,242,355,241.25","11,322,248,551.25","10,828,705,711.25"


In [16]:
# Find the intersection of minimum and maximum of dates across tickers
# then use this dataframe for all observations

mindate = dtdf.groupby('Ticker')['Date'].min()
maxdate = dtdf.groupby('Ticker')['Date'].max()

begdate = max(mindate)
enddate = min(maxdate)

print begdate, enddate

2004-09-17 00:00:00 2017-06-02 00:00:00


In [17]:
df1 = dtdf.where((dtdf['Date'] >= begdate) & (dtdf['Date'] <= enddate))\
      .dropna(axis=0, how='all')
display(df1)

,Date,Ticker,Y,R01,R02,R03,R04,R05,R06,R07,R08,R09,R10,AV01,AV02,AV03,AV04,AV05,AV06,AV07,AV08,AV09,AV10
1174,2004-09-17,AAPL,1.00,0.15,0.14,0.15,0.15,0.18,0.15,0.15,0.18,0.18,0.14,"3,319,415,143.77","3,236,033,626.71","3,258,304,381.56","3,312,884,518.84","3,313,902,602.14","3,252,700,631.16","3,171,795,198.34","3,154,013,565.75","3,129,196,884.37","3,183,216,021.42"
1175,2004-09-20,AAPL,1.00,0.21,0.15,0.14,0.15,0.15,0.18,0.15,0.15,0.18,0.18,"3,403,315,337.75","3,319,415,143.77","3,236,033,626.71","3,258,304,381.56","3,312,884,518.84","3,313,902,602.14","3,252,700,631.16","3,171,795,198.34","3,154,013,565.75","3,129,196,884.37"
1176,2004-09-21,AAPL,1.00,0.22,0.21,0.15,0.14,0.15,0.15,0.18,0.15,0.15,0.18,"3,396,841,597.73","3,403,315,337.75","3,319,415,143.77","3,236,033,626.71","3,258,304,381.56","3,312,884,518.84","3,313,902,602.14","3,252,700,631.16","3,171,795,198.34","3,154,013,565.75"
1177,2004-09-22,AAPL,1.00,0.22,0.22,0.21,0.15,0.14,0.15,0.15,0.18,0.15,0.15,"3,481,614,209.56","3,396,841,597.73","3,403,315,337.75","3,319,415,143.77","3,236,033,626.71","3,258,304,381.56","3,312,884,518.84","3,313,902,602.14","3,252,700,631.16","3,171,795,198.34"
1178,2004-09-23,AAPL,1.00,0.16,0.22,0.22,0.21,0.15,0.14,0.15,0.15,0.18,0.15,"3,517,572,661.93","3,481,614,209.56","3,396,841,597.73","3,403,315,337.75","3,319,415,143.77","3,236,033,626.71","3,258,304,381.56","3,312,884,518.84","3,313,902,602.14","3,252,700,631.16"
1179,2004-09-24,AAPL,1.00,0.13,0.16,0.22,0.22,0.21,0.15,0.14,0.15,0.15,0.18,"3,493,829,640.35","3,517,572,661.93","3,481,614,209.56","3,396,841,597.73","3,403,315,337.75","3,319,415,143.77","3,236,033,626.71","3,258,304,381.56","3,312,884,518.84","3,313,902,602.14"
1180,2004-09-27,AAPL,1.00,0.08,0.13,0.16,0.22,0.22,0.21,0.15,0.14,0.15,0.15,"3,251,931,587.17","3,493,829,640.35","3,517,572,661.93","3,481,614,209.56","3,396,841,597.73","3,403,315,337.75","3,319,415,143.77","3,236,033,626.71","3,258,304,381.56","3,312,884,518.84"
1181,2004-09-28,AAPL,1.00,0.09,0.08,0.13,0.16,0.22,0.22,0.21,0.15,0.14,0.15,"3,271,469,465.79","3,251,931,587.17","3,493,829,640.35","3,517,572,661.93","3,481,614,209.56","3,396,841,597.73","3,403,315,337.75","3,319,415,143.77","3,236,033,626.71","3,258,304,381.56"
1182,2004-09-29,AAPL,1.00,0.11,0.09,0.08,0.13,0.16,0.22,0.22,0.21,0.15,0.14,"3,346,371,872.54","3,271,469,465.79","3,251,931,587.17","3,493,829,640.35","3,517,572,661.93","3,481,614,209.56","3,396,841,597.73","3,403,315,337.75","3,319,415,143.77","3,236,033,626.71"
1183,2004-09-30,AAPL,1.00,0.12,0.11,0.09,0.08,0.13,0.16,0.22,0.22,0.21,0.15,"3,316,268,998.65","3,346,371,872.54","3,271,469,465.79","3,251,931,587.17","3,493,829,640.35","3,517,572,661.93","3,481,614,209.56","3,396,841,597.73","3,403,315,337.75","3,319,415,143.77"


In [19]:
# Convert to "wide format" so that for each day, 
# there are a number of columns,
# and each ticekr's name is atteched to its respective columns
df2 = df1.pivot(index='Date', columns='Ticker')
df2.columns = [' '.join(col).strip() for col in df2.columns.values]
df2.reset_index(inplace=True)

display(df2)

,Date,Y AAPL,Y AMZN,Y GOOGL,Y IBM,Y MSFT,R01 AAPL,R01 AMZN,R01 GOOGL,R01 IBM,R01 MSFT,R02 AAPL,R02 AMZN,R02 GOOGL,R02 IBM,R02 MSFT,R03 AAPL,R03 AMZN,R03 GOOGL,R03 IBM,R03 MSFT,R04 AAPL,R04 AMZN,R04 GOOGL,R04 IBM,R04 MSFT,R05 AAPL,R05 AMZN,R05 GOOGL,R05 IBM,R05 MSFT,R06 AAPL,R06 AMZN,R06 GOOGL,R06 IBM,R06 MSFT,R07 AAPL,R07 AMZN,R07 GOOGL,R07 IBM,R07 MSFT,R08 AAPL,R08 AMZN,R08 GOOGL,R08 IBM,R08 MSFT,R09 AAPL,R09 AMZN,R09 GOOGL,R09 IBM,R09 MSFT,R10 AAPL,R10 AMZN,R10 GOOGL,R10 IBM,R10 MSFT,AV01 AAPL,AV01 AMZN,AV01 GOOGL,AV01 IBM,AV01 MSFT,AV02 AAPL,AV02 AMZN,AV02 GOOGL,AV02 IBM,AV02 MSFT,AV03 AAPL,AV03 AMZN,AV03 GOOGL,AV03 IBM,AV03 MSFT,AV04 AAPL,AV04 AMZN,AV04 GOOGL,AV04 IBM,AV04 MSFT,AV05 AAPL,AV05 AMZN,AV05 GOOGL,AV05 IBM,AV05 MSFT,AV06 AAPL,AV06 AMZN,AV06 GOOGL,AV06 IBM,AV06 MSFT,AV07 AAPL,AV07 AMZN,AV07 GOOGL,AV07 IBM,AV07 MSFT,AV08 AAPL,AV08 AMZN,AV08 GOOGL,AV08 IBM,AV08 MSFT,AV09 AAPL,AV09 AMZN,AV09 GOOGL,AV09 IBM,AV09 MSFT,AV10 AAPL,AV10 AMZN,AV10 GOOGL,AV10 IBM,AV10 MSFT
0,2004-09-17,1.00,0.00,1.00,0.00,1.00,0.15,0.08,0.07,0.01,-0.00,0.14,0.10,0.06,0.03,0.01,0.15,0.12,0.05,0.03,0.02,0.15,0.11,0.05,0.03,0.01,0.18,0.06,0.08,0.06,0.03,0.15,0.04,0.08,0.03,-0.00,0.15,0.02,0.08,0.01,-0.01,0.18,0.09,0.07,0.02,0.01,0.18,0.09,0.07,0.01,0.00,0.14,0.10,0.07,-0.01,0.01,"3,319,415,143.77","302,882,886.14","3,250,494,371.22","334,924,552.41","1,258,988,245.34","3,236,033,626.71","296,993,529.92","3,174,982,857.13","338,041,871.96","1,290,779,159.90","3,258,304,381.56","294,829,499.08","3,221,311,366.90","332,998,633.04","1,296,191,499.53","3,312,884,518.84","276,458,134.97","3,208,938,125.76","334,178,917.69","1,293,082,539.13","3,313,902,602.14","269,322,060.58","3,305,496,494.59","337,757,136.48","1,285,899,782.33","3,252,700,631.16","276,929,220.24","3,476,070,785.80","350,802,459.20","1,281,088,341.37","3,171,795,198.34","280,958,269.06","3,480,190,122.43","353,405,892.48","1,276,240,177.37","3,154,013,565.75","287,074,085.06","3,478,905,651.54","347,334,354.66","1,283,501,046.39","3,129,196,884.37","292,031,919.35","3,514,978,060.75","345,455,604.45","1,292,588,395.44","3,183,216,021.42","302,333,536.62","3,528,430,379.53","352,557,530.11","1,332,626,186.15"
1,2004-09-20,1.00,0.00,1.00,1.00,1.00,0.21,0.11,0.17,0.01,0.02,0.15,0.08,0.07,0.01,-0.00,0.14,0.10,0.06,0.03,0.01,0.15,0.12,0.05,0.03,0.02,0.15,0.11,0.05,0.03,0.01,0.18,0.06,0.08,0.06,0.03,0.15,0.04,0.08,0.03,-0.00,0.15,0.02,0.08,0.01,-0.01,0.18,0.09,0.07,0.02,0.01,0.18,0.09,0.07,0.01,0.00,"3,403,315,337.75","295,295,629.57","1,016,160,618.87","341,530,128.68","1,286,011,701.53","3,319,415,143.77","302,882,886.14","3,250,494,371.22","334,924,552.41","1,258,988,245.34","3,236,033,626.71","296,993,529.92","3,174,982,857.13","338,041,871.96","1,290,779,159.90","3,258,304,381.56","294,829,499.08","3,221,311,366.90","332,998,633.04","1,296,191,499.53","3,312,884,518.84","276,458,134.97","3,208,938,125.76","334,178,917.69","1,293,082,539.13","3,313,902,602.14","269,322,060.58","3,305,496,494.59","337,757,136.48","1,285,899,782.33","3,252,700,631.16","276,929,220.24","3,476,070,785.80","350,802,459.20","1,281,088,341.37","3,171,795,198.34","280,958,269.06","3,480,190,122.43","353,405,892.48","1,276,240,177.37","3,154,013,565.75","287,074,085.06","3,478,905,651.54","347,334,354.66","1,283,501,046.39","3,129,196,884.37","292,031,919.35","3,514,978,060.75","345,455,604.45","1,292,588,395.44"
2,2004-09-21,1.00,0.00,1.00,1.00,1.00,0.22,0.10,0.10,0.01,0.01,0.21,0.11,0.17,0.01,0.02,0.15,0.08,0.07,0.01,-0.00,0.14,0.10,0.06,0.03,0.01,0.15,0.12,0.05,0.03,0.02,0.15,0.11,0.05,0.03,0.01,0.18,0.06,0.08,0.06,0.03,0.15,0.04,0.08,0.03,-0.00,0.15,0.02,0.08,0.01,-0.01,0.18,0.09,0.07,0.02,0.01,"3,396,841,597.73","301,833,067.24","955,933,543.65","341,112,924.52","1,293,635,728.00","3,403,315,337.75","295,295,629.57","1,016,160,618.87","341,530,128.68","1,286,011,701.53","3,319,415,143.77","302,882,886.14","3,250,494,371.22","334,924,552.41","1,258,988,245.34","3,236,033,626.71","296,993,529.92","3,174,9

In [20]:
# this will form the basis for the machine learning part of the anaysis
df2.to_csv('Predict_01.CSV')